# Ulsan Namgu(남구) Network using Folium

## UNIST School of Natural Science

### Mathematical Science Major

### 20121229 JunPyo Park

About Folium?

<img src="https://trello-attachments.s3.amazonaws.com/59103d52b56a24582f00dc97/5ac9878e1e1550dea50a1f5b/adbdfd2fff8aa3075bfa1bbb72335221/image.png"></img>

Github Link : https://github.com/python-visualization/folium

## Results

<img src="https://trello-attachments.s3.amazonaws.com/59103d52b56a24582f00dc97/5ac9878e1e1550dea50a1f5b/75f543ac2aa1d5d05a984636c9846188/image.png"></img>

<img src="https://trello-attachments.s3.amazonaws.com/59103d52b56a24582f00dc97/5ac9878e1e1550dea50a1f5b/fe9ed5b7f61d04ec42bd0656c8178cc3/image.png"></img>

## Codes

In [63]:
import os
import folium
import pandas as pd
import numpy as np

Read processed Data for whole Ulsan

In [64]:
nodes = pd.read_csv('ulsan_nodes.csv')
links = pd.read_csv('ulsan_links.csv')

In [65]:
nodes = nodes[['Id','NODE_NAME','STNL_REG','latitude','longitude']]
nodes.head()

,Id,NODE_NAME,STNL_REG,latitude,longitude
0,1920003500,사례교,192,35.594499,129.347704
1,1920003600,사례교,192,35.595572,129.347603
2,1920002100,-,192,35.564435,129.327960
3,1920002600,동중 삼거리,192,35.574493,129.338874
4,1920002400,중구청 삼거리,192,35.569200,129.331570


Table for STNL_REG code

<img src="https://trello-attachments.s3.amazonaws.com/59103d52b56a24582f00dc97/5ac8b77017464fe59d4b728e/6388818a77f59b02e011068d58dc8145/image.png"></img>

I choose '울산광역시 남구' , STNL_REG == 193

Taking namgu_nodes

In [66]:
namgu_nodes = nodes[nodes['STNL_REG']==193]
namgu_id = dict()
for i in namgu_nodes['Id'] :
    namgu_id[i] = 0
namgu_nodes.head()

,Id,NODE_NAME,STNL_REG,latitude,longitude
426,1930003400,태화교,193,35.549644,129.308352
427,1930003900,태화교,193,35.552367,129.308968
428,1930000300,SK합성수지앞,193,35.502741,129.311231
429,1930000400,두왕사거리,193,35.503334,129.308730
430,1930000500,상개삼거리,193,35.504289,129.316423


In [67]:
links = links[['Source','Target']]
links.head()

,Source,Target
0,1920013900,1920015600
1,1920015600,1920013900
2,1920015600,1920015800
3,1920015800,1920015600
4,1920006400,1920037700


We shoud remove the links which is not connected in 남구

In [68]:
source_in = links['Source'].apply(lambda x : x in namgu_id) # check Sources are in namgu_id
target_in = links['Target'].apply(lambda x : x in namgu_id) # check Targets are in namgu_id
# source_in and target_in are boolean type pandas.Series which contains True or False

In [69]:
namgu_links = links[source_in & target_in] # contain if both target and source are contained in namgu_id
namgu_links.head()

,Source,Target
1142,1930003900,1930003400
1327,1930002309,1930054300
1328,1930054300,1930002309
1329,1930036100,1930034500
1330,1930059900,1930059500


By checking the length, we can find that links are reduced from 8065 to 2241

In [74]:
print(len(links))
print(len(namgu_links))

8065
2241


Calculating Degree of each namgu_nodes

In [58]:
for ix, row in namgu_links.iterrows():
    namgu_id[row['Source']] += 1
    namgu_id[row['Target']] += 1

In [75]:
# Positioning the Standard Point for out Folium Map
std_point = tuple(nodes[nodes['NODE_NAME']=='구암초교'][['latitude','longitude']].iloc[0])
std_point

(35.5233723996693, 129.33730157655654)

## Draw Node

In [61]:
map_osm = folium.Map(location=std_point, zoom_start=13)

for ix, row in namgu_nodes.iterrows():
    location = (row['latitude'], row['longitude'])
    folium.Circle(
        location=location,
        radius=namgu_id[row['Id']] * 10,
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        popup=row['NODE_NAME']
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)
# it takes some time.....
map_osm

## Draw Links

In [62]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in namgu_links.iterrows():
    start = tuple(namgu_nodes[namgu_nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(namgu_nodes[namgu_nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm